In [30]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torchvision
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
from torch.utils.data import random_split
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision import models

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [31]:
root_dir = "/home/adityaraut/Documents/Machine-Learning-FCC/dl_fcc/archive01"

In [32]:
train_df = pd.read_csv('/home/adityaraut/Documents/Machine-Learning-FCC/dl_fcc/archive01/train.csv')
val_df = pd.read_csv('/home/adityaraut/Documents/Machine-Learning-FCC/dl_fcc/archive01/val.csv')

In [34]:
train_df.shape, val_df.shape

((1034, 2), (133, 2))

In [35]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

In [23]:
class CustomDataset(Dataset):
    
    def __init__(self, dataframe, transform, root_dir):
        self.dataframe = dataframe
        self.transform = transform
        self.label = dataframe['category'].values
        self.root_dir = root_dir
        
    def __len__(self):
        return self.dataframe.shape[0]
    
    def __getitem__(self, idx):
        img_rel_path = self.dataframe.iloc[idx, 0]
        img_path = os.path.join(self.root_dir, img_rel_path)
        label = int(self.label[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [10]:
train_dataset = CustomDataset(dataframe=train_df, transform= transform, root_dir=root_dir)
val_dataset = CustomDataset(dataframe=val_df, transform= transform, root_dir=root_dir)

In [11]:
lr = 0.001
batch_size = 32
epochs = 15

In [12]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [120]:
googlenet_model = models.googlenet(weights='DEFAULT')

In [121]:
for params in googlenet_model.parameters():
    params.requires_grad = True

In [122]:
googlenet_model.fc

Linear(in_features=1024, out_features=1000, bias=True)

In [123]:
num_classes = len(train_df['category'].unique())

In [124]:
googlenet_model.fc = torch.nn.Linear(googlenet_model.fc.in_features, num_classes)

In [125]:
googlenet_model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [126]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(googlenet_model.parameters(), lr=lr, weight_decay = 1e-4)

In [127]:
total_loss_train_plot = []
total_acc_train_plot = []
total_loss_val_plot = []
total_acc_val_plot = []

for epoch in range(epochs):
    
    # --- Training ---
    googlenet_model.train()
    total_loss_train = 0
    total_acc_train = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        train_pred = googlenet_model(inputs)
        loss = criterion(train_pred, labels)
        loss.backward()
        optimizer.step()
        
        total_loss_train += loss.item()
        total_acc_train += (train_pred.argmax(1) == labels).sum().item()
    
    train_loss = total_loss_train / len(train_loader)
    train_acc = total_acc_train / len(train_dataset) * 100
    
    total_loss_train_plot.append(round(train_loss, 4))
    total_acc_train_plot.append(round(train_acc, 4))
    
    # --- Validation ---
    googlenet_model.eval()
    val_loss_total = 0
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            val_pred = googlenet_model(inputs)
            loss = criterion(val_pred, labels)
            val_loss_total += loss.item()
            val_correct += (val_pred.argmax(1) == labels).sum().item()
    
    val_loss = val_loss_total / len(val_loader)
    val_acc = val_correct / len(val_dataset) * 100
    
    total_loss_val_plot.append(round(val_loss, 4))
    total_acc_val_plot.append(round(val_acc, 4))
    
    # --- Print summary ---
    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


Epoch 1/15 | Train Loss: 0.4541, Train Acc: 80.95% | Val Loss: 0.2647, Val Acc: 90.98%
Epoch 2/15 | Train Loss: 0.2084, Train Acc: 94.39% | Val Loss: 0.1470, Val Acc: 91.73%
Epoch 3/15 | Train Loss: 0.1150, Train Acc: 96.23% | Val Loss: 0.2070, Val Acc: 92.48%
Epoch 4/15 | Train Loss: 0.0594, Train Acc: 98.26% | Val Loss: 0.2375, Val Acc: 91.73%
Epoch 5/15 | Train Loss: 0.0305, Train Acc: 98.94% | Val Loss: 0.4310, Val Acc: 93.98%
Epoch 6/15 | Train Loss: 0.0757, Train Acc: 97.87% | Val Loss: 0.3136, Val Acc: 91.73%
Epoch 7/15 | Train Loss: 0.1210, Train Acc: 95.74% | Val Loss: 0.8435, Val Acc: 85.71%
Epoch 8/15 | Train Loss: 0.1081, Train Acc: 96.62% | Val Loss: 0.1968, Val Acc: 91.73%
Epoch 9/15 | Train Loss: 0.0649, Train Acc: 97.87% | Val Loss: 0.1717, Val Acc: 93.98%
Epoch 10/15 | Train Loss: 0.0541, Train Acc: 98.65% | Val Loss: 0.1261, Val Acc: 96.24%
Epoch 11/15 | Train Loss: 0.0138, Train Acc: 99.61% | Val Loss: 0.1110, Val Acc: 96.24%
Epoch 12/15 | Train Loss: 0.0400, Train A

In [141]:
googlenet_model = models.googlenet(weights='DEFAULT')
for param in googlenet_model.parameters():
    param.requires_grad = False

googlenet_model.fc = torch.nn.Linear(googlenet_model.fc.in_features, num_classes)
googlenet_model.fc.requires_grad_(True)
googlenet_model.to(device)

optimizer = torch.optim.Adam(googlenet_model.fc.parameters(), lr=1e-4)


In [142]:
total_loss_train_plot = []
total_acc_train_plot = []
total_loss_val_plot = []
total_acc_val_plot = []

for epoch in range(epochs):
    
    # --- Training ---
    googlenet_model.train()
    total_loss_train = 0
    total_acc_train = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        train_pred = googlenet_model(inputs)
        loss = criterion(train_pred, labels)
        loss.backward()
        optimizer.step()
        
        total_loss_train += loss.item()
        total_acc_train += (train_pred.argmax(1) == labels).sum().item()
    
    train_loss = total_loss_train / len(train_loader)
    train_acc = total_acc_train / len(train_dataset) * 100
    
    total_loss_train_plot.append(round(train_loss, 4))
    total_acc_train_plot.append(round(train_acc, 4))
    
    # --- Validation ---
    googlenet_model.eval()
    val_loss_total = 0
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            val_pred = googlenet_model(inputs)
            loss = criterion(val_pred, labels)
            val_loss_total += loss.item()
            val_correct += (val_pred.argmax(1) == labels).sum().item()
    
    val_loss = val_loss_total / len(val_loader)
    val_acc = val_correct / len(val_dataset) * 100
    
    total_loss_val_plot.append(round(val_loss, 4))
    total_acc_val_plot.append(round(val_acc, 4))
    
    # --- Print summary ---
    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


Epoch 1/15 | Train Loss: 1.0899, Train Acc: 39.36% | Val Loss: 1.0149, Val Acc: 51.13%
Epoch 2/15 | Train Loss: 1.0233, Train Acc: 48.74% | Val Loss: 0.9690, Val Acc: 60.90%
Epoch 3/15 | Train Loss: 0.9717, Train Acc: 58.90% | Val Loss: 0.8946, Val Acc: 68.42%
Epoch 4/15 | Train Loss: 0.9143, Train Acc: 65.96% | Val Loss: 0.8484, Val Acc: 66.92%
Epoch 5/15 | Train Loss: 0.8788, Train Acc: 68.86% | Val Loss: 0.8335, Val Acc: 69.92%
Epoch 6/15 | Train Loss: 0.8398, Train Acc: 70.89% | Val Loss: 0.7863, Val Acc: 74.44%
Epoch 7/15 | Train Loss: 0.8187, Train Acc: 73.02% | Val Loss: 0.7694, Val Acc: 74.44%
Epoch 8/15 | Train Loss: 0.7756, Train Acc: 74.66% | Val Loss: 0.7404, Val Acc: 74.44%
Epoch 9/15 | Train Loss: 0.7563, Train Acc: 75.44% | Val Loss: 0.7143, Val Acc: 75.94%
Epoch 10/15 | Train Loss: 0.7349, Train Acc: 76.11% | Val Loss: 0.7213, Val Acc: 76.69%
Epoch 11/15 | Train Loss: 0.7241, Train Acc: 75.05% | Val Loss: 0.6738, Val Acc: 74.44%
Epoch 12/15 | Train Loss: 0.7043, Train A